# Dynamic Analysis

> Dynamic analyses collect information about the system as it executes

Opposed to **static analysis**: 

> Static analyses analyze the system’s artifacts to obtain information that is valid for all possible executions (e.g, program structure or potential calls between different modules). 
(View-Driven Software Architecture Reconstruction...)


### Uses in Architecture Recovery
In the Extraction phase: 
  - dependencies between components (e.g. `Model` -> `UI`)
  - properties of components (e.g. `Model` is never used, `connection` is slow, etc.)

## Prerequisite: Running the System

- not that trivial as you might think
- challenges
  - configuration
  - dependencies
  - unwritten rules
  - some systems don't have a clear entry point (e.g. libraries)
- helpful practices
  - continuous integration
  - containerization
  - infrastructure as code
  
  

## How to Get the Program Running? 

- install required 3rd party librarie
- deploy using containers
- ...

## Which Scenarios to Run from the System?

- Run the unit tests if they exist 
- Exercise "features" 

> A feature is a realized functional requirement of a system. [...] an observable unit of behavior of a system triggered by the user [Eisenbarth et al., 2003].

[Eisenbarth et al., 2003]. Thomas Eisenbarth, Rainer Koschke, and Daniel Simon. Locating features in source code. IEEE Computer, 29(3):210–224, March 2003.

## Approaches

- logging
- instrumentation
- traffic analysis


## Approach #1: Logging

- invasive - adding logging statements in the program
  - implies changing the program
- allows surgical precision - adding log statements only where relevant


## Approach #2: Instrumentation

What to instrument: 
- source code
  - using reflection, or code generation
- binaries
  - e.g. virtual machine instrumentation


### Instrumenting Binaries. e.g. Java

![](./images/java_instrumentation.png)

- Java programs are compiled into bytecode
- Bytecode is executed on the JVM
- You can provide a Java Agent (via command line argument -javaagent) that modifieds the bytecode before it being executed


Advantages:
  - no need for parsing
  - works for multiple languages



  
More on this topic:
- https://blog.sqreen.com/building-a-dynamic-instrumentation-agent-for-java/


### Instrumenting Source Code

- can be done using **reflection**

> Reflection is the ability of a program to manipulate as data something representing the state of the program during its own execution. 
> - **Introspection** is the ability for a program to observe and therefore reason about its own state. 
> - **Intercession** is the ability for a program to modify its own execution state or alter its own interpretation or meaning.

- in some languages it's easier to do (e.g. Ruby, Python, Java)  than in others



### Example: Introspection in Python

Goal: 
- a program to observe it's own state (e.g. a class observing it's own methods)


Python Specific: 
- use the `cls_name.__dict__.items( )` to get all the attributes of a class and filter those which represent a method because they have the `__call__` annotation

In [ ]:
# an object-oriented foobar example
class Foo(object):

    def __init__(self):
        self.x= 'foo'

    def do(self):
        print(self.x)


class Bar(object):

    def __init__(self, foo):
        self.foo = foo

    def do(self):
        self.foo.do()

In [ ]:
def methods_in_class(cls_name):
    """ list all methods in a class"""
    result = {}
    for key, value in cls_name.__dict__.items( ):
            if hasattr( value, '__call__' ):
                result [key] = value
    return result


In [ ]:
methods_in_class(Foo)

Notes:
- it's the same program, even if it's in three cells
  - could have moved the `list_methods` in the Bar class

### Example: Intercession in Python

Goal: 
- let's have our program replace it's methods on the fly 
  - each with another method that prints a note when the function is entered
  - we will thus trace the execution of the program!


We rely on `setattr( cls_name, key, replacement )` to replace the method found under the name `key` with `replacement`




In [ ]:
# same class as before
class Foo(object):

    def __init__(self):
        self.x= 'foo'

    def do(self):
        print(self.x)


class Bar(object):

    def __init__(self, foo):
        self.foo = foo

    def do(self):
        self.foo.do()

In [ ]:
def replace_methods( cls_name, replacement ):
    """ replace every method in class cls_name with a wrapper method """
    for key, value in methods_in_class(cls_name).items():
            setattr( cls_name, key, replacement( value ) )
            
def wrapper( fn ):
    def result( *args, **kwargs ):
        print (f'entered {fn}')
        return fn( *args, **kwargs )
    return result


In [ ]:
replace_methods(Foo, wrapper)
Foo().do()

##### Notes:
- this is easier in a dynamically typed language
- we have used **function wrappers**, a design pattern where:
  - a function *wraps* another function in order to
    - perform some *prologue* and/or *epilogue* tasks
    - optimize (e.g. cache results )
  - while the *wrapper* is *fully* compatible with the wrapped function so it can be used instead



More on Function Wrappers: 
- https://wiki.python.org/moin/FunctionWrappers
- Wrappers to the Rescue: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.18.6550&rep=rep1&type=pdf

### Example: Tracing Method Calls with Function Wrappers

- we want a wrapper that prints out when a method is called
  - the method name
  - who called it
  

- by deploying this in selected parts of our system we can trace all method calls



In [ ]:
class Foo(object):

    def __init__(self):
        self.x= 'foo'

    def do(self):
        print(self.x)


class Bar(object):

    def __init__(self, foo):
        self.foo = foo

    def do(self):
        self.foo.do()

In [ ]:
import inspect
import sys

def trace_call(caller_method, called_method):
    print(caller_method + " -> " + called_method)
    
def tracing_wrapper( cls, fn ):
    def result( *args, **kwargs ):
        caller_method =  inspect.stack()[1].frame.f_code.co_name    
        called_method = str(cls) + "." + fn.__name__
        trace_call(caller_method, called_method)
        return fn( *args, **kwargs )
    return result

def wrap_methods( cls, wrapper ):
    """ replace every method in class cls_name with a wrapper method """
    for key, value in cls.__dict__.items( ):
        if hasattr( value, '__call__' ):
            setattr( cls, key, wrapper( cls, value ) )


In [ ]:
wrap_methods(Foo, tracing_wrapper)
wrap_methods(Bar, tracing_wrapper)
Bar(Foo()).do()

#### And to Our Case Study Now!


In [ ]:
%cd /Users/mircea/Zeeguu-Core

In [ ]:
from zeeguu_core.model import User

In [ ]:
from tools.past_exercises_for_user import past_exercises_for
past_exercises_for(534)

In [ ]:
def all_classes_in(mod):
    import inspect, importlib
    """ return all the classes in a given module """
    result = []
    for name, thingy in inspect.getmembers(importlib.import_module(mod)):
        if inspect.ismodule(thingy):
            if thingy.__name__.startswith(mod):
                result.extend(all_classes_in(thingy.__name__))

        elif inspect.isclass(thingy):
            if (thingy.__module__ == mod):
                result.append(thingy)
    return result


In [ ]:
import zeeguu_core
all_classes_in('zeeguu_core')

In [ ]:
for each in all_classes_in('zeeguu_core'):
    wrap_methods(each, tracing_wrapper)
    

In [ ]:
past_exercises_for(534)

####  Challenges for you: Improve this if you can!
- fully qualified names of the caller method
- log to file
- compute overhead
- extract graph from unit tests
- compare dyanmically extracted graph with statically extracted graph



### Disadvantages of Wrappers
- they introduce an overhead (but then, so do all code related tracing)
- they require you to obtain the **live** objects (must be in the same process as the instrumented code)


### Advantages of Wrappers

- still allows surgical precision 
- allow **even more surgical deployment and removal** of wrappers at runtime
- as opposed to off-the-shelf tools that trace the entire execution of the program
  - compare with
    ```python -m trace --trackcalls past_exercises_for_user.py ```
    
    - (executed from within the tools folder)
    
    
More on the `trace` module: https://docs.python.org/3/library/trace.html

## Approach #3: Traffic Analysis

- useful for service oriented architectures
- monitors the messages on the wire
- powerful approach for reverse engineering services


Read: https://danlebrero.com/2017/04/06/documenting-your-architecture-wireshark-plantuml-and-a-repl/
- If somebody wants to work on this as their report, replicating this for Zeeguu-API / Zeeguu-Web would be great!
- If not for now, doing something like this would be a great starting point for a thesis



## Limitations

- limited by execution coverage 
  - a program does not reach an execution point... => no data (e.g. Word but user never prints)
  
- can slow down the application considerably 


- can result in a large amount of of data

  


## Uses Beyond Architecture Recovery

- Performance monitoring (e.g. the FMD)
- Intercepting and tracing specific calls
  - e.g. calls to the DB, calls across the network
  
- Quality control (e.g. test coverage tool)
- Dynamic optimizations 
- Logging Energy Usage (https://help.apple.com/instruments/mac/current/#/dev03a7149d)

### Further Reading

Papers: 
- **Visualizing the Execution of Java Programs**. Wim De Pauw, Erik Jensen, Nick Mitchell, Gary Sevitsky, John Vlissides, Jeaha Yang

- **Correlating Features and Code Using a Compact Two-Sideed Trace Analysis Approach**. Orla Greevy, Stephane Ducasse. 
